In [1]:
from itertools import cycle
from random import randint,choice
from copy import deepcopy

In [2]:
BLACK = "BLACK"
WHITE = "WHITE"
HOLE  = "HOLE"
UP    = "UP"
DOWN  = "DOWN"
RIGHT = "RIGHT"
LEFT  = "LEFT"
COLORS     = [BLACK, WHITE]
DIRECTIONS = [UP, DOWN, RIGHT, LEFT]

In [3]:
# 完全ランダム
def chimpan_algo(game):
    stones = game.on_player.stones
    stone = choice(stones)
    direction = choice(DIRECTIONS)
    return stone, direction

In [4]:
# 自殺点防止
def gorilla_algo(game):
    prev_stones = game.on_player.stones
    candidates = []
    for prev_stone in prev_stones:
        for direction in DIRECTIONS:
            dummy_game = deepcopy(game)
            dummy_stones = dummy_game.on_player.stones
            for dummy_stone in dummy_stones:
                if dummy_stone.i == prev_stone.i and dummy_stone.j == prev_stone.j:
                    candidate_stone = dummy_stone
            candidate_i = candidate_stone.i
            candidate_j = candidate_stone.j
            candidate_stone.push(direction)
            next_stones = dummy_game.on_player.stones
            if len(prev_stones) == len(next_stones):
                for stone in prev_stones:
                    if stone.i == candidate_i and stone.j == candidate_j:
                        candidates.append((stone, direction))
            
    if len(candidates) == 0:
        return chimpan_algo(game)
    else:
        return choice(candidates)

In [5]:
# 相手を落とせる時は落とす
def utan_algo(game):
    prev_stones = game.on_player.stones
    candidates = []
    for prev_stone in prev_stones:
        for direction in DIRECTIONS:
            dummy_game = deepcopy(game)
            dummy_stones = dummy_game.on_player.stones
            for dummy_stone in dummy_stones:
                if dummy_stone.i == prev_stone.i and dummy_stone.j == prev_stone.j:
                    candidate_stone = dummy_stone
            candidate_i = candidate_stone.i
            candidate_j = candidate_stone.j
            competitor = next(dummy_game.players)
            pre_comp_stones = len(competitor.stones)
            candidate_stone.push(direction)
            next_comp_stones = len(competitor.stones)
            next_stones = dummy_game.on_player.stones
            if pre_comp_stones > next_comp_stones:
                for stone in prev_stones:
                    if stone.i == candidate_i and stone.j == candidate_j:
                        candidates.append((stone, direction))
            
    if len(candidates) == 0:
        return gorilla_algo(game)
    else:
        return choice(candidates)

In [6]:
class Board:
    def __init__(self, observer):
        self.observer = observer
        self.observer.add(self)
        self.board = [[None for j in range(5)] for i in range(5)]
        for i, line in enumerate(self.board):
            for j, cell in enumerate(line):
                if i == 0:
                    line[j] = Stone(BLACK, i, j, self.observer)
                elif i == 4:
                    line[j] = Stone(WHITE, i, j, self.observer)
                elif i == 2 and j == 2:
                    line[j] = Stone(HOLE, i, j, self.observer)
    
    def __str__(self):
        text = []
        for line in self.board:
            text.append(" | ".join([str(cell) if cell else "　" for cell in line ]))
        return "\n-----------------------\n".join(text)
    
    def show(self):
        print(self)

In [7]:
class Stone:
    
    def __init__(self, color, i, j, observer):
        self.observer = observer
        self.observer.add(self)
        self.color = color
        self.i = i
        self.j = j
        self._droped = False
    
    def push(self, direction):
        prev_i = self.i
        prev_j = self.j
        if direction == UP:
            self.i -= 1
        elif direction == DOWN:
            self.i += 1
        elif direction == RIGHT:
            self.j += 1
        elif direction == LEFT:
            self.j -= 1
        self.observer.push(prev_i, prev_j, self, direction)
    
    @property
    def droped(self):
        return self._droped
    
    @droped.setter
    def droped(self, value):
        self._droped = value
        self.observer.drop_stone(self)
    
    def __str__(self):
        if self.color == BLACK:
            return "■"
        elif self.color == WHITE:
            return "□"
        elif self.color == HOLE:
            return "○"
        

In [8]:
class Player:
    def __init__(self, observer, color, algo):
        self.observer = observer
        self.observer.add(self)
        self.color = color
        self.algo = algo
        self._point = 0
        self.win = False
        self.stones = []
        self._set_stones()
    
    def _set_stones(self):
        for stone in self.observer.stones:
            if stone.color == self.color or stone.color == HOLE:
                self.stones.append(stone)
    
    def play(self, game):
        stone, direction = self.algo(game)
        stone.push(direction)
    
    @property
    def point(self):
        return self._point
    
    @point.setter
    def point(self, value):
        self._point = value
        if value == 2:
            self.win = True

In [9]:
class Observer:
    
    def __init__(self):
        self.stones  = []
        self.players = []
        self.on_player = None
    
    def add(self, observed):
        if isinstance(observed, Board):
            self.board = observed
        elif isinstance(observed, Stone):
            self.stones.append(observed)
        elif isinstance(observed, Player):
            self.players.append(observed)
    
    def push(self, prev_i, prev_j, stone, direction):
        self.board.board[prev_i][prev_j] = None
        i = stone.i
        j = stone.j
        if 0 <= i <= 4 and 0 <= j <= 4:
            target = self.board.board[i][j]
            if target:
                if target.color == HOLE:
                    stone.droped = True
                    return
                else:
                    target.push(direction)
            self.board.board[i][j] = stone
        else:
            stone.droped = True
    
    def drop_stone(self, stone):
        for player in self.players:
            if stone in player.stones:
                player.stones.remove(stone)
                if stone.color != HOLE:
                    self.add_point(stone)
    
    def add_point(self, stone):
        if stone.color != self.on_player.color:
            self.on_player.point += 1

In [10]:
class Ostle:
    def __init__(self,black_algo=chimpan_algo, white_algo=chimpan_algo):
        self.observer = Observer()
        self.board = Board(self.observer)
        self.black_player = Player(self.observer, BLACK, algo=black_algo)
        self.white_player = Player(self.observer, WHITE, algo=white_algo)
        self.players = cycle([self.black_player, self.white_player])
        self.on_player = next(self.players)
        self.gameover = False
        self.winner = None
    
    def turn(self,show=False):
        if len(self.on_player.stones) == 0:
            self.gameover = True
            return
        self.on_player.play(self)
        if show:
            print(self.on_player.color)
            self.board.show()
            print("")
            print("")
        self.isGameover()
        self.on_player = next(self.players)
    
    def isGameover(self):
        isWin = self.on_player.win
        noStone = not(self.on_player.color in [stone.color for stone in self.on_player.stones])
        noChoice = not(self.on_player.stones)
        if isWin or noStone or noChoice:
            self.gameover = True
            if isWin:
                self.winner = self.on_player
            elif noStone:
                self.winner = next(self.players)
                next(self.players)
        
    
    def play_all(self, show=False):
        while not(self.gameover):
            self.turn(show)
    
    def show(self):
        self.board.show()
        
    
    
    @property
    def on_player(self):
        return self._on_player
    
    @on_player.setter
    def on_player(self, value):
        self._on_player = value
        self.observer.on_player = value

In [11]:
# 初期化
# chimpan_algo, gorilla_algo, utan_algoから、
# 黒と白それぞれの戦略アルゴリズムを決定
game = Ostle(black_algo=chimpan_algo, white_algo=gorilla_algo)

In [12]:
# 1ターンずつプレイ
for i in range(10):
    game.turn(show=True)

BLACK
■ | 　 | ■ | ■ | ■
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | ○ | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
□ | □ | □ | □ | □


WHITE
■ | 　 | ■ | ■ | ■
-----------------------
　 | 　 | ○ | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
□ | □ | □ | □ | □


BLACK
■ | ■ | ■ | 　 | ■
-----------------------
　 | 　 | ○ | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
□ | □ | □ | □ | □


WHITE
■ | ■ | ■ | 　 | ■
-----------------------
　 | 　 | ○ | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | □
-----------------------
□ | □ | □ | □ | 　


BLACK
■ | ■ | 　 | 　 | ■
-----------------------
　 | 　 | ○ | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | □
-----------------------
□ | □ | □ | □ | 　


WHITE
■ | ■ | 　 | 　 | ■
------

In [13]:
#  最後までプレイ
game.play_all(show=True)

BLACK
　 | ■ | ■ | 　 | ■
-----------------------
　 | ○ | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | □
-----------------------
□ | □ | □ | □ | 　


WHITE
　 | ■ | ■ | 　 | ■
-----------------------
　 | ○ | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | □
-----------------------
□ | □ | 　 | □ | □


BLACK
　 | 　 | ■ | 　 | ■
-----------------------
　 | ○ | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | □
-----------------------
□ | □ | 　 | □ | □


WHITE
　 | 　 | ■ | 　 | ■
-----------------------
　 | ○ | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | □ | □
-----------------------
□ | □ | 　 | 　 | □


BLACK
　 | 　 | ■ | 　 | ■
-----------------------
○ | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | 　 | 　
-----------------------
　 | 　 | 　 | □ | □
-----------------------
□ | □ | 　 | 　 | □


WHITE
　 | 　 | ■ | 　 | ■
------

In [14]:
def trial(N,black_algo,white_algo):
    result = {BLACK:0, WHITE:0, "DRAW":0}
    for i in range(N):
        game = Ostle(black_algo=black_algo, white_algo=white_algo)
        game.play_all()
        if game.winner:
            winner = game.winner.color
            result[game.winner.color] += 1
        else:
            result["DRAW"] += 1
    print("%s vs %s" %(black_algo.__name__.split("_")[0], white_algo.__name__.split("_")[0] ))
    print(result)

In [15]:
# 各アルゴリズム同士を100回対戦させた結果
algos = [chimpan_algo, gorilla_algo, utan_algo]
for black_algo in algos:
    for white_algo in algos:
        trial(100, black_algo, white_algo)
        print("------------------------------")

chimpan vs chimpan
{'BLACK': 51, 'WHITE': 49, 'DRAW': 0}
------------------------------
chimpan vs gorilla
{'BLACK': 14, 'WHITE': 86, 'DRAW': 0}
------------------------------
chimpan vs utan
{'BLACK': 7, 'WHITE': 89, 'DRAW': 4}
------------------------------
gorilla vs chimpan
{'BLACK': 85, 'WHITE': 13, 'DRAW': 2}
------------------------------
gorilla vs gorilla
{'BLACK': 55, 'WHITE': 45, 'DRAW': 0}
------------------------------
gorilla vs utan
{'BLACK': 4, 'WHITE': 96, 'DRAW': 0}
------------------------------
utan vs chimpan
{'BLACK': 81, 'WHITE': 10, 'DRAW': 9}
------------------------------
utan vs gorilla
{'BLACK': 98, 'WHITE': 2, 'DRAW': 0}
------------------------------
utan vs utan
{'BLACK': 46, 'WHITE': 54, 'DRAW': 0}
------------------------------
